In [2]:
from IPython.display import clear_output
import pyrealsense2 as rs
import numpy as np
import cv2
from IPython.display import Image
from PIL import Image as PILImage
import io
import time
import sys
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.idl.default import unitree_go_msg_dds__SportModeState_
from unitree_sdk2py.idl.unitree_go.msg.dds_ import SportModeState_
from unitree_sdk2py.go2.sport.sport_client import (
    SportClient,
    PathPoint,
    SPORT_PATH_POINT_SIZE,
)
from unitree_sdk2py.go2.obstacles_avoid.obstacles_avoid_client import ObstaclesAvoidClient
import math
from dataclasses import dataclass
import matplotlib.pyplot as plt

%matplotlib inline

## get camera feed and show live

In [3]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
Object_colors = list(np.random.rand(80,3)*255)
# Start pipeline
pipeline.start(config)

In [4]:
while True:
    # Wait for a frame from the RealSense camera
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    clear_output(wait=True)
    
    # plt.imshow(color_image)
    # plt.show()
    # # print(color_image.shape)
    # time.sleep(0.5)
    

KeyboardInterrupt: 

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)

finally:

    # Stop streaming
    pipeline.stop()

In [ ]:
def yolo5_object_detection():
    # Initialize YOLOv5 model (ensure the model file is available)
    # model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 small model
    # model = Object_detector

    # Configure RealSense pipeline
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
    Object_colors = list(np.random.rand(80,3)*255)
    # Start pipeline
    pipeline.start(config)
    cnt = 0
    flag = 1
    movement = 0.3
    try:
        while True:
            
            # Wait for a frame from the RealSense camera
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()

            if not color_frame:
                continue

            # Convert RealSense frame to numpy array
            color_image = np.asanyarray(color_frame.get_data())

            # Use GStreamer (through OpenCV) to resize the image
            frame = resized_image = cv2.resize(color_image, (640, 480))

            # Perform YOLOv5 detection
            # objs = results = model(resized_image)
            objs = results = Object_detector.detect(resized_image)

            for obj in objs:
                print("="*5, obj)
                print("done")
                label = obj["label"]
                score = obj["score"]
                
                
                print("1")
                [(xmin, ymin), (xmax, ymax)] = obj["bbox"]
                print("2")
                color = Object_colors[Object_classes.index(label)]
                print("3")
                frame = cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                print("4")
                frame = cv2.putText(frame,
                        f"{label} ({str(score )})",
                        (xmax, ymax),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.75,
                        color,
                        1,
                        cv2.LINE_AA,
                    )
                print(f"{label} ({str(score )})")
                if score > 0.40:
                    time.sleep(.5)
                    current_time = datetime.now()
                    if label == 'chair' and (not time1 or ((current_time - time1) > 1)):
                        time1 = datetime.strptime("12:30", "%H:%M")
                        # sport_client.StandUp()
                        sport_client.Hello()
                        flag = -1
                        # sport_client.BalanceStand()
                        # obsclient.UseRemoteCommandFromApi(True)
                        # obsclient.Move(0,0,0.3)
                        # time.sleep(1)
                        # obsclient.Move(0,0,0)
                        # obsclient.UseRemoteCommandFromApi(False)
                        # movement = -movement
                        # if flag == 1:
                        #     print("standing up")
                        #     sport_client.StandUp()
                        #     flag = -1
                        # else:
                        #     print("standing down")
                        #     sport_client.StandDown()
                        #     flag = 1
                    # elif label == 'bottle':
                    #     robot.left(0.2)
                    # elif label == 'tv':
                    #     robot.backward(0.2)
                    elif label == 'person':
                        pass
                        # sport_client.StandUp()
                        # flag = -1
                        # sport_client.BalanceStand()
                        # sport_client.Move(movement,0,0)
                        # movement = -movement
                    
            ipython_image = Image(data=cv2.imencode(".jpeg", frame)[1])  # Replace with your image file path

            # Convert IPython Image to a format OpenCV can use
            # 1. Get image bytes
            image_bytes = ipython_image.data
            
            # 2. Convert bytes to a PIL image
            pil_image = PILImage.open(io.BytesIO(image_bytes))
            
            # 3. Convert PIL image to a NumPy array
            opencv_image = np.array(pil_image)
            
            # 4. Convert RGB to BGR (OpenCV uses BGR format)
            opencv_image = cv2.cvtColor(opencv_image, cv2.COLOR_RGB2BGR)
            
            # Show the image using OpenCV
            # cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            # cv2.imshow("RealSense", opencv_image)
            cv2.imwrite(f"{cnt}.jpg", opencv_image)
            cnt += 1

            
            
            # cv2.imshow('RealSense', cv2.imencode(".jpeg", frame)[1])
            # display(Image(data=cv2.imencode(".jpeg", frame)[1]))
            clear_output(wait=True)
            time.sleep(5)

            # # Render the detection results
            # detections = results.pandas().xyxy[0]  # Extract detections in Pandas format
            # for _, row in detections.iterrows():
            #     x1, y1, x2, y2, conf, cls = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']), row['confidence'], row['name']
            #     label = f"{cls} {conf:.2f}"
            #     cv2.rectangle(resized_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            #     cv2.putText(resized_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # # Display the resulting frame
            # cv2.imshow('YOLOv5 Object Detection', resized_image)

            # # Exit on pressing 'q'
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #     break

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Stop the pipeline and release resources
    
        pipeline.stop()
        cv2.destroyAllWindows()

# Call the function to run the detection
yolo5_object_detection()